In [ ]:
import torch
import torch.nn as nn
import time
import numpy as np
import random

class FNN(nn.Module):
    '''Full-Connected Neural Network with Batch Normalization'''
    def __init__(self, input_dim: int, num_hiddens, output_dim: int, activate_fun, device=torch.device("cpu")):
        super(FNN, self).__init__()
        self.num_hiddens = num_hiddens
        self.activate_fun = activate_fun
        self.input_dim = input_dim
        self.output_dim = output_dim

        self.dense_layers = nn.ModuleList()
        self.dense_layers.append(nn.Linear(input_dim, num_hiddens[0]))

        # 添加隐藏层
        for i in range(len(num_hiddens) - 1):
            self.dense_layers.append(nn.Linear(num_hiddens[i], num_hiddens[i+1]))
        
        self.dense_layers.append(nn.Linear(num_hiddens[-1], output_dim))

        self.to(device)

    def forward(self, x):
        for i in range(len(self.dense_layers) - 1):
            x = self.dense_layers[i](x)
            x = self.activate_fun(x)
        x = self.dense_layers[-1](x)
        return x


def set_seed(seed):
    # set random seed
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

set_seed(2024)

# default start time is 0
T = 1 # total time
d = 50 # dimension
n_time = 100 # time interval number

r_eq = 0.1
sigma_eq = 0.3
d_eq = 0.1

lam = 0.1    # jump activity
delta = 0.5   # jump size

h = T/n_time # time interval step
n_sample = 128
time_grid = torch.linspace(0, T, n_time + 1)

x0 = torch.pi/2

device = 'cuda'

def sample_path(n_sample, d, n_time, h, lam, delta, device = 'cpu'):
    sqrth = torch.sqrt(torch.tensor(h))
    
    dW_sample = torch.zeros(n_sample, d, n_time)
    Jumps_sample = torch.zeros(n_sample, d, n_time)
    
    for i in range(n_time):
        dW_sample[:, :, i] = torch.normal(mean=0.0, std=1.0, size=(n_sample, d)) * sqrth
        P_sample = torch.poisson(torch.full((n_sample * d,), lam * h))
        Jumps_sample[:, :, i] = torch.tensor(
            [torch.sum(torch.empty(int(k)).uniform_(-delta, delta)) for k in P_sample]
        ).reshape(n_sample, d)
    
    dW_sample = dW_sample.to(device)
    Jumps_sample = Jumps_sample.to(device)
    return dW_sample, Jumps_sample

def f(t, X, Y, Z, U):
    # nonlinear term
    term1 = - r_eq * Y 
    term2 = 0.5 * torch.exp(-3 * r_eq * (T - t) )* sigma_eq**2 * (d_eq * torch.sum(torch.sin(X), dim=1, keepdim=True))**3
    result = term1 + term2 + U
    return result

def g(t, X):
    # terminal conditions
    output = torch.exp(- r_eq * (T - t)) * d_eq * torch.sum(torch.sin(X), dim=1, keepdim=True)
    return output

modelList_Z = [FNN(input_dim=d, 
                   num_hiddens=[128, 128], 
                   output_dim = d, 
                   activate_fun=nn.ReLU(),
                   device=device) 
               for i in range(n_time-1)]
modelList_U = [FNN(input_dim=d, 
                   num_hiddens=[128, 128], 
                   output_dim = 1, 
                   activate_fun=nn.ReLU(),
                   device=device) 
               for i in range(n_time-1)]

y_init = torch.nn.Parameter(torch.empty(1, 1,device=device).uniform_(3, 4))
z_init = torch.nn.Parameter(torch.empty(1, d,device=device).uniform_(-0.1, 0.1))
u_init = torch.nn.Parameter(torch.empty(1, 1,device=device).uniform_(-0.1, 0.1))

def loss_fun(dW_sample, Jumps_sample):
    X = torch.ones((n_sample,d),device=device) * x0
    Y = torch.ones((n_sample,1),device=device) * y_init
    Z = torch.ones((n_sample,1),device=device) * z_init
    U = torch.ones((n_sample,1),device=device) * u_init

    for i in range(n_time):
        Y = Y - f(time_grid[i], X, Y, Z, U) * h
        Y = Y + sigma_eq * Y * torch.sum(Z * dW_sample[:, :, i],dim=1,keepdim=True) + U * h
        X = X + sigma_eq * Y * dW_sample[:, :, i] + Jumps_sample[:, :, i]

        if i < n_time - 1:
            Z = modelList_Z[i](X)
            U = modelList_U[i](Z * Jumps_sample[:, :, i])

    term_delta = Y - g(time_grid[i+1], X) 
    clamp_term_delta = torch.clamp(term_delta, -50, 50)
    loss = torch.mean(clamp_term_delta**2)
    
    return loss

params = [y_init, z_init, u_init] 
for model in modelList_Z:
    params += list(model.parameters())
for model in modelList_U:
    params += list(model.parameters())


optimizer = torch.optim.Adam(params, lr=0.002)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.8)


truth_value = g(torch.zeros((1,1)), torch.ones((1,d))*torch.pi/2)
print(truth_value)

start_time = time.time()
train_his = []

for iter in range(1500):
    dW_sample, Jumps_sample = sample_path(n_sample, d, n_time, h, lam, delta,device=device)  
    optimizer.zero_grad()
    loss = loss_fun(dW_sample, Jumps_sample)
    loss.backward()
    optimizer.step()
    scheduler.step()
    if (iter+1) % 10 == 0 or iter == 0:
        g_error = torch.mean((torch.abs((truth_value-y_init.detach().cpu())/truth_value)))

        print(f'iter:{iter+1}, loss:{loss.item():.4e}, time:{time.time()-start_time:.2f}, y0:{torch.mean(y_init).item():.4f}, error:{g_error:.4f},lr:{optimizer.param_groups[0]["lr"]:.4f}')
        train_his.append({'iter':iter+1,'loss': loss.item(), 'y0': torch.mean(y_init).item(), 'time': time.time()-start_time, 'error':g_error,'lr':optimizer.param_groups[0]["lr"]})
        

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(train_his)
df['iter'] = df.index*10
# 输出DataFrame
print(df.describe())

In [ ]:
plt.plot(df['iter'],df['loss'])
plt.show()

In [ ]:
# 绘制损失、y0
plt.plot(df['iter'],df['error'])
plt.show()

In [ ]:
step_size = 300
df_sampled = df[df['iter'] % step_size == 0]

summary_df = pd.DataFrame({
    'Iteration': df_sampled['iter'],
    'Loss': df_sampled['loss'],
    'y0': df_sampled['y0'],
    'error': df_sampled['error'],
})
latex_table = summary_df.to_latex(index=False, float_format="%.4f", column_format='cccc', header=True)

print(latex_table)